# Calibrate Cameras
[https://docs.opencv.org/3.4/d9/dab/tutorial_homography.html](https://docs.opencv.org/3.4/d9/dab/tutorial_homography.html)

In [2]:
from models.dataset.dataset_interface import DatasetInterface
from matplotlib import pyplot as plt
from pathlib import Path
import numpy as np
import cv2

In [3]:
resource_path = Path("../../resources")
uncal_path = resource_path / "images/uncalibrated"
cal_path = resource_path / "images/calibrated"
dir_to_calibrate = Path("c_dataset_d_1")

dataset_interface = DatasetInterface(uncal_path / dir_to_calibrate)

calibrated_dc = DatasetInterface(cal_path / dir_to_calibrate)

sift = cv2.SIFT_create()
index_params = dict(algorithm = 1, trees = 5)
search_params = dict(checks=50)

for rs_rgb, rs_depth, zv_rgb, zv_depth in dataset_interface:
    zv_kp, zv_des = sift.detectAndCompute(zv_rgb, None)
    rs_kp, rs_des = sift.detectAndCompute(rs_rgb, None)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(zv_des, rs_des, k=2)

    #-- Filter matches using the Lowe's ratio test
    ratio_thresh = 0.7
    good_matches = [m for m,n in matches if m.distance < ratio_thresh * n.distance]

    #-- Draw matches
    # img_matches = np.empty((max(zv_rgb.shape[0], rs_rgb.shape[0]), zv_rgb.shape[1] + zv_rgb.shape[1], 3), dtype=np.uint8)
    # cv2.drawMatches(zv_rgb, zv_kp, rs_rgb, rs_kp, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    #-- Localize the object
    obj = np.empty((len(good_matches),2), dtype=np.float32)
    scene = np.empty((len(good_matches),2), dtype=np.float32)
    for i in range(len(good_matches)):
        #-- Get the keypoints from the good matches
        obj[i,0] = zv_kp[good_matches[i].queryIdx].pt[0]
        obj[i,1] = zv_kp[good_matches[i].queryIdx].pt[1]
        scene[i,0] = rs_kp[good_matches[i].trainIdx].pt[0]
        scene[i,1] = rs_kp[good_matches[i].trainIdx].pt[1]
    H, _ =  cv2.findHomography(obj, scene, cv2.RANSAC)
    H_inv = np.linalg.inv(H)

    #-- Get the corners from the image_1 ( the object to be "detected" )
    # obj_corners = np.empty((4,1,2), dtype=np.float32)
    # obj_corners[0,0,0] = 0
    # obj_corners[0,0,1] = 0
    # obj_corners[1,0,0] = zv_rgb.shape[1]
    # obj_corners[1,0,1] = 0
    # obj_corners[2,0,0] = zv_rgb.shape[1]
    # obj_corners[2,0,1] = zv_rgb.shape[0]
    # obj_corners[3,0,0] = 0
    # obj_corners[3,0,1] = zv_rgb.shape[0]
    # scene_corners = cv2.perspectiveTransform(obj_corners, H)
    # #-- Draw lines between the corners (the mapped object in the scene - image_2 )
    # cv2.line(img_matches, (int(scene_corners[0,0,0] + zv_rgb.shape[1]), int(scene_corners[0,0,1])),\
    #     (int(scene_corners[1,0,0] + zv_rgb.shape[1]), int(scene_corners[1,0,1])), (0,255,0), 4)
    # cv2.line(img_matches, (int(scene_corners[1,0,0] + zv_rgb.shape[1]), int(scene_corners[1,0,1])),\
    #     (int(scene_corners[2,0,0] + zv_rgb.shape[1]), int(scene_corners[2,0,1])), (0,255,0), 4)
    # cv2.line(img_matches, (int(scene_corners[2,0,0] + zv_rgb.shape[1]), int(scene_corners[2,0,1])),\
    #     (int(scene_corners[3,0,0] + zv_rgb.shape[1]), int(scene_corners[3,0,1])), (0,255,0), 4)
    # cv2.line(img_matches, (int(scene_corners[3,0,0] + zv_rgb.shape[1]), int(scene_corners[3,0,1])),\
    #     (int(scene_corners[0,0,0] + zv_rgb.shape[1]), int(scene_corners[0,0,1])), (0,255,0), 4)
    # #-- Show detected matches

    rs_rgb_warp = cv2.warpPerspective(rs_rgb, H_inv, (zv_rgb.shape[1], zv_rgb.shape[0]))
    rs_depth_warp = cv2.warpPerspective(rs_depth, H_inv, (zv_rgb.shape[1], zv_rgb.shape[0]))

    calibrated_dc.append_and_save(rs_rgb_warp, rs_depth_warp, zv_rgb, zv_depth)

    # debug output
    # plt.figure(2)
    # plt.imshow(zv_rgb)

    # plt.figure(3)
    # plt.imshow(rs_rgb_warp)

    # img_both = cv2.addWeighted(zv_rgb, 0.5, rs_rgb_warp, 0.5, 0)
    # plt.figure(4)
    # plt.imshow(img_both)

AssertionError: 